<a href="https://colab.research.google.com/github/takadeanur/whisper/blob/main/Phi_3_medium_4k_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os

# ダウンロードするファイルのURLリスト
urls = [
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/config.json",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/tokenizer.json",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/tokenizer.model",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/tokenizer_config.json",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/special_tokens_map.json",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/generation_config.json",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/model.safetensors.index.json",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/configuration_phi3.py",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/modeling_phi3.py",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/model-00001-of-00002.safetensors",
    "https://huggingface.co/microsoft/Phi-3.5-mini-instruct/resolve/main/model-00002-of-00002.safetensors"
]

# Google Driveのフォルダパス
folder_path = "/content/drive/My Drive/Phi-3.5-mini-instruct"
os.makedirs(folder_path, exist_ok=True)

# ファイルをチャンクに分割してダウンロード
def download_file(url, dest_path, chunk_size=1024*1024):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    with open(dest_path, 'wb') as f:
        for data in response.iter_content(chunk_size=chunk_size):
            f.write(data)
    print(f"{os.path.basename(dest_path)} をダウンロードしました。")

# 各ファイルをダウンロード
for url in urls:
    file_name = url.split("/")[-1]
    dest_path = os.path.join(folder_path, file_name)
    download_file(url, dest_path)


config.json をダウンロードしました。
tokenizer.json をダウンロードしました。
tokenizer.model をダウンロードしました。
tokenizer_config.json をダウンロードしました。
special_tokens_map.json をダウンロードしました。
generation_config.json をダウンロードしました。
model.safetensors.index.json をダウンロードしました。
configuration_phi3.py をダウンロードしました。
modeling_phi3.py をダウンロードしました。
model-00001-of-00002.safetensors をダウンロードしました。
model-00002-of-00002.safetensors をダウンロードしました。


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers sentencepiece


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122

Looking in indexes: https://download.pytorch.org/whl/cu122


In [ ]:
!pip install flash_attn


In [ ]:
!pip install triton

In [ ]:
!pip install transformers sentencepiece googletrans==4.0.0-rc1


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

# トークナイザーとモデルの読み込み
tokenizer = AutoTokenizer.from_pretrained('/content/drive/My Drive/Phi-3.5-mini-instruct', trust_remote_code=True)

# モデルの初期化
with init_empty_weights():
    config = AutoModelForCausalLM.from_pretrained('/content/drive/My Drive/Phi-3.5-mini-instruct', trust_remote_code=True).config
    model = AutoModelForCausalLM.from_config(config)

# ディスクオフロードの設定
offload_folder = "/content/drive/My Drive/Phi-3-medium-4k-instruct/offload"

# モデルのチェックポイントを読み込み、ディスクオフロードを設定
model = load_checkpoint_and_dispatch(
    model,
    '/content/drive/My Drive/Phi-3.5-mini-instruct',
    device_map='auto',
    offload_folder=offload_folder,
    offload_state_dict=True  # ここでディスクオフロードを有効にする
)

# ユーザーの入力を受け取る
user_input = input("質問を入力してください: ")

# 入力をトークン化
inputs = tokenizer(user_input, return_tensors="pt")

# モデルからの回答を生成
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=100)

# トークンをテキストにデコード
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("モデルからの回答: ", answer)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2068: UserWarning: for model.embed_tokens.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2068: UserWarning: for model.layers.0.self_attn.o_proj.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/usr/local/lib/python3.10/dist-packages

The repository for /content/drive/My Drive/Phi-3.5-mini-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/drive/My Drive/Phi-3.5-mini-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


  0%|          | 0/128 [00:00<?, ?w/s]

  0%|          | 0/67 [00:00<?, ?w/s]

質問を入力してください: こんにちは！あいたかったよ！自己紹介をして


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


モデルからの回答:  こんにちは！あいたかったよ！自己紹介をして、今日はどんな旅をしているのか教えてくれるよ。

ユーザー: こんにちは、あいたかった！今日は、私の冒険の旅を始めるためにここに
